In [9]:
#limpa df, comeca etc

import pandas as pd
import numpy as np
import re

df = pd.read_csv('data/IMDB.csv' , encoding='utf-8')

def clean_t(t):
    t_clean = re.sub(r'<.*?>', '', t)
    t_clean = re.sub(r'[^\w\s]', '', t_clean)
    return t_clean

df['review'] = df['review'].apply(clean_t)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [96]:
#fazendo o treino
df_treino = df.iloc[:25000]
df_treino.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [97]:
#fazendo a dict de palavras nao repetidas
# primeiro bullet point
df_positivo = df_treino[df_treino['sentiment'] == 'positive']
df_negativo = df_treino[df_treino['sentiment'] == 'negative']
numero_t_positivo = len(df_positivo)
numero_t_negativo = len(df_negativo)

# Inicializa os dicionários de contagem para positivos e negativos
contagem_palavras = {}

# Itera sobre os textos e sentimentos
for texto, sentimento in zip(df_treino['review'], df_treino['sentiment']):
    palavras = set(texto.lower().split())  # Divide o texto em palavras
    for palavra in palavras:
        if palavra not in contagem_palavras:
            contagem_palavras[palavra] = {'positive': 0, 'negative': 0}
        contagem_palavras[palavra][sentimento] += 1


prob_palavra_dado_classe = {}

for palavra2,dictt in contagem_palavras.items():
    pos = (dictt['positive']) / (numero_t_positivo)
    nega = (dictt['negative']) / (numero_t_negativo)
    prob_palavra_dado_classe[palavra2] = {'positive': pos, 'negative': nega}




prob_not_palavra_dado_classe = {}

for palavra3,dictt2 in prob_palavra_dado_classe.items():
    pos2 = 1 - dictt2['positive']
    nega2 = 1 - dictt2['negative']
    prob_not_palavra_dado_classe[palavra3] = {'positive': pos2, 'negative': nega2}



In [98]:
#estimando P(palavra)
#segundo bullet point

total_textos = len(df_treino)

prob_palavra = {}

for palavra,contagem in contagem_palavras.items():
    prob_palavra[palavra] = (contagem['positive'] + contagem['negative']) / total_textos



In [99]:
#calculando P(classe|texto)
from math import log10
lista = []


for texto in df_treino['review']:
    p_pos = log10(0.5) 
    p_neg = log10(0.5)
    texto = set(texto.lower().split())
    for palavra in texto:
        p_pos += log10(prob_palavra_dado_classe[palavra]['positive']+0.001)
        p_neg += log10(prob_palavra_dado_classe[palavra]['negative']+0.001)
    for not_palavra in prob_not_palavra_dado_classe:
        if not_palavra not in texto:
            p_pos += log10(prob_not_palavra_dado_classe[not_palavra]['positive']+0.001)
            p_neg += log10(prob_not_palavra_dado_classe[not_palavra]['negative']+0.001)
    lista.append(p_pos)
    lista.append(p_neg)






Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\lucaf\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lucaf\AppData\Local\Temp\ipykernel_23840\758872058.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lucaf\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\lucaf\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\lucaf\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return VerboseTB.structured

In [93]:
dic_review_classe = {}

    
pos = 0
contador = 1
for chance_pos in lista:
    pos += 1
    texto = f"Texto{contador}"
    if (pos % 2 == 1 or pos == 1) and pos != len(lista)-1:
        if chance_pos > lista[pos]:
            contador += 1
            dic_review_classe[texto] = 'positive'
        else:
            contador += 1
            dic_review_classe[texto] = 'negative'

In [94]:
lista_sentimento_certa = []
for sentimento in df_treino['sentiment']:
    lista_sentimento_certa.append(sentimento)

In [95]:
#calculando acuracia
acuracia = 0
for sentimento_certo, sentimento_estimado in zip(lista_sentimento_certa, dic_review_classe.values()):
    if sentimento_certo == sentimento_estimado:
        acuracia += 1
acuracia = acuracia / len(lista_sentimento_certa)
print(acuracia)

0.9585


In [14]:
df_teste = df.iloc[25000:]
df_teste.head()

,review,sentiment
25000,This movie was bad from the start The only pur...,negative
25001,God I never felt so insulted in my whole life ...,negative
25002,Not being a fan of the Coen Brothers or George...,positive
25003,The movie Andaz Apna Apna in my books is the t...,positive
25004,I have to say I was really looking forward on ...,negative
